In [29]:
from yfpy.query import YahooFantasySportsQuery
from dotenv import find_dotenv, load_dotenv
import os
import pandas as pd
import numpy as np

In [30]:
yahoo_query = YahooFantasySportsQuery(
    auth_dir='auth/',
    league_id='1411473',
)

In [31]:
cereal = yahoo_query.get_league_standings().serialized()

2022-11-15 09:47:27.920 - WARNING - query.py - yfpy.query:790 - No game id or season/year provided, defaulting to current fantasy season.


In [32]:
l_standings = dict()
for team in cereal['teams']:
    team_info = team['team'].serialized()
    name = team_info['name'].decode()
    standings = team_info['team_standings']
    wins = standings['outcome_totals']['wins']
    losses = standings['outcome_totals']['losses']
    l_standings[name] = {'wins': wins, 'losses': losses}

In [33]:
# standings_pd = pd.DataFrame(l_standings).head()
# standings_pd['name'] = standings_pd['name'].str.decode("utf-8")

In [34]:
matchups = []
for week in range(11,14):
    for matchup in yahoo_query.get_league_matchups_by_week(week):
        matchup_teams = matchup['matchup'].serialized()['teams']
        matchups.append(
            [
                matchup_teams[0]['team'].serialized()['name'].decode(),
                matchup_teams[1]['team'].serialized()['name'].decode(),
            ]
        )

2022-11-15 09:47:28.850 - WARNING - query.py - yfpy.query:790 - No game id or season/year provided, defaulting to current fantasy season.
2022-11-15 09:47:30.698 - WARNING - query.py - yfpy.query:790 - No game id or season/year provided, defaulting to current fantasy season.
2022-11-15 09:47:31.307 - WARNING - query.py - yfpy.query:790 - No game id or season/year provided, defaulting to current fantasy season.


In [35]:
from copy import deepcopy

In [36]:
l_standings

{'Belichicken Tenders': {'wins': 9, 'losses': 1},
 'Tyler to be named later': {'wins': 8, 'losses': 2},
 'Sophkillaaaaaaaaa': {'wins': 7, 'losses': 3},
 "Rachel's Peerless Team": {'wins': 7, 'losses': 3},
 "Zach's FUKU Fighters": {'wins': 6, 'losses': 4},
 "Steven's Slam-Dunk Team": {'wins': 5, 'losses': 5},
 "Ethan Rosenblum's Nifty Team": {'wins': 5, 'losses': 5},
 'McCaf(r)é': {'wins': 4, 'losses': 6},
 'Greg the Leg': {'wins': 4, 'losses': 6},
 'LROZ': {'wins': 3, 'losses': 7},
 'Flour Power': {'wins': 1, 'losses': 9},
 'Intern Season': {'wins': 1, 'losses': 9}}

In [44]:
final_results = pd.DataFrame()
for i in range(10000):
    l_standings_sim = deepcopy(l_standings)
    for matchup in matchups:
        winner = np.random.randint(0,2)
        win_name = matchup[winner]
        lose_name = [i for i in matchup if i != win_name][0]
        l_standings_sim[win_name]['wins'] += 1
        l_standings_sim[lose_name]['losses'] += 1
        
    sim_standings = pd.DataFrame(l_standings_sim).transpose().sample(frac=1)
    sim_standings['rank'] = sim_standings['wins'].rank(method='first', ascending=False)
    sim_standings['playoffs'] = np.where(
        sim_standings['rank'] <= 6,
        1,
        0
    )
    sim_standings['bye'] = np.where(
        sim_standings['rank'] <= 2,
        1,
        0
    )

    final_results = pd.concat([final_results, sim_standings])

In [45]:
len(final_results)

120000

In [46]:
final_results_names = final_results.reset_index()

In [47]:
final_results_names = final_results_names.groupby('index')[['playoffs', 'bye']].mean().reset_index()

In [48]:
final_results_names.head()

,index,playoffs,bye
0,Belichicken Tenders,1.000,0.9098
1,Ethan Rosenblum's Nifty Team,0.451,0.0005
2,Flour Power,0.000,0.0000
3,Greg the Leg,0.137,0.0000
4,Intern Season,0.000,0.0000


In [49]:
final_results_names = final_results_names.sort_values('playoffs', ascending = False)
final_results_names['playoffs'] = np.round(final_results_names['playoffs']*100, 1).astype(float).astype(str) + '%'
final_results_names['bye'] = np.round(final_results_names['bye']*100, 1).astype(float).astype(str) + '%'

In [50]:
final_results_names.columns = ['Team Name', 'Playoff %', 'Bye %']
final_results_names

,Team Name,Playoff %,Bye %
0,Belichicken Tenders,100.0%,91.0%
10,Tyler to be named later,100.0%,65.1%
7,Rachel's Peerless Team,98.4%,21.9%
8,Sophkillaaaaaaaaa,97.3%,19.3%
11,Zach's FUKU Fighters,85.3%,2.7%
9,Steven's Slam-Dunk Team,45.8%,0.0%
1,Ethan Rosenblum's Nifty Team,45.1%,0.0%
3,Greg the Leg,13.7%,0.0%
6,McCaf(r)é,12.1%,0.0%
5,LROZ,2.4%,0.0%
